In [2]:
import os
import pydeck

import pandas as pd
import geopandas

import matplotlib as mpl
import pydeck

In [3]:
MAX_BUILDINGS = 6000

In [4]:
start_point = (6.1402,46.208742)

In [20]:
delta = 0.001
bbox =  (start_point[0] - delta, start_point[1]-delta, start_point[0] + delta, start_point[1]+delta) 

In [7]:
folder = fr'E:\switchdrive\Institution\p3'

filepath = os.path.join(folder, 'bases.gpkg')
bases = geopandas.read_file(filepath)

filepath = os.path.join(folder, 'facades.gpkg')
facade = geopandas.read_file(filepath)

filepath = os.path.join(folder, 'toits.gpkg')
toits = geopandas.read_file(filepath)

bases.columns = [s.lower() for s in bases.columns]
facade.columns = [s.lower() for s in facade.columns]
toits.columns = [s.lower() for s in toits.columns]

In [21]:
# filepath = './data/ge/bases.gpkg'
# bases = geopandas.read_file(filepath, bbox=bbox)

# filepath = './data/ge/facades.gpkg'
# facade = geopandas.read_file(filepath, bbox=bbox)

# filepath = './data/ge/toits.gpkg'
# toits = geopandas.read_file(filepath, bbox=bbox)


# bases.columns = [s.lower() for s in bases.columns]
# facade.columns = [s.lower() for s in facade.columns]
# toits.columns = [s.lower() for s in toits.columns]

In [7]:
# TODO can make a lookup of buildings which is shorter with just EGID and point, using geopandas
# select buildings using the BBOX. Then have an in-memory 'DB' of geoms - just a dict keyed with egid
# so you can select geoms just by subselecting from the dict

In [8]:
MAPBOX_API_KEY = os.environ["MAPBOX_API_KEY"]

KeyError: 'MAPBOX_API_KEY'

In [9]:
heat_demand = pd.read_csv('data/ge_heat_demand.csv')

In [47]:
bases = bases.merge(heat_demand, on='egid')

norm = mpl.colors.LogNorm(vmin=1, vmax=bases.q_heat_total.max())
cmap = mpl.cm.get_cmap('plasma')

cmap_val = cmap(norm(bases.q_heat_total)) 

bases['color'] = (cmap_val* 255).astype(int).tolist()

In [48]:
toits = toits.merge(heat_demand, on='egid')

norm = mpl.colors.LogNorm(vmin=1, vmax=toits.q_heat_total.max())
cmap = mpl.cm.get_cmap('plasma')

cmap_val = cmap(norm(toits.q_heat_total)) 

toits['color'] = (cmap_val* 255).astype(int).tolist()


In [49]:
facade = facade.merge(heat_demand, on='egid')

norm = mpl.colors.LogNorm(vmin=1, vmax=facade.q_heat_total.max())
cmap = mpl.cm.get_cmap('plasma')

cmap_val = cmap(norm(facade.q_heat_total)) 

facade['color'] = (cmap_val* 255).astype(int).tolist()


In [13]:
vals = []
def _f(*args, **kwargs):
    global vals
    vals.append((args, kwargs))

In [30]:

def filtered_buildings_geom(bbox):
    return (bases.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]],
            facade.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]],
            toits.cx[bbox[0]: bbox[2], bbox[1]:bbox[3]])

In [29]:
f_base, f_face, f_toit = filtered_buildings_geom(bbox)

In [71]:
coords = f_base.centroid.map(lambda p: [p.x, p.y])

INITIAL_VIEW_STATE = pydeck.data_utils.compute_view(coords.values)
INITIAL_VIEW_STATE.zoom = 16
INITIAL_VIEW_STATE.max_zoom = 16

/var/folders/23/f82vmc8x4d18zwnnbl6dxvnw0000gn/T/ipykernel_82656/1168186431.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = f_base.centroid.map(lambda p: [p.x, p.y])


In [75]:
# AWS Open Data Terrain Tiles
TERRAIN_IMAGE = "https://s3.amazonaws.com/elevation-tiles-prod/terrarium/{z}/{x}/{y}.png"

# Define how to parse elevation tiles
ELEVATION_DECODER = {"rScaler": 256, "gScaler": 1, "bScaler": 1 / 256, "offset": -32768}

SURFACE_IMAGE = f"https://api.mapbox.com/v4/mapbox.satellite/{{z}}/{{x}}/{{y}}@2x.png?access_token={MAPBOX_API_KEY}"


# lights = pydeck.LightSettings(
#         lights_position=[start_point[0], start_point[1], 8000, -3.807751, 54.104682, 8000], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


# lights = pydeck.LightSettings(
#     number_of_lights=1,
#         lights_position=[start_point[0]+1, start_point[1]+1, 80], ambient_ratio=0.4, diffuse_ratio=0.6,
#     lights_strength=0.1
#     )


terrain_layer = pydeck.Layer(
    "TerrainLayer", elevation_decoder=ELEVATION_DECODER, texture=SURFACE_IMAGE, elevation_data=TERRAIN_IMAGE,
    # light_settings=lights

)


bases_layer = pydeck.Layer('GeoJsonLayer', 
                           data=f_base, get_fill_color=[255,100,255], pickable=True
                          )
facade_layer = pydeck.Layer('GeoJsonLayer', 
                            data=f_face, 
                            get_fill_color='color', 
                            line_width_scale=0.1,
                            pickable=True,
                            # stroked=False,
                            filled=True,
                            wireframe=True,
                            # light_settings=lights

                           )
# facade_layer = pydeck.Layer('GeoJsonLayer', data=facade, get_fill_color='color', pickable=True)
toits_layer = pydeck.Layer('GeoJsonLayer', 
                           data=f_toit, 
                           get_fill_color='color', 
                           pickable=True,
                           stroked=False,
                           filled=True,
                           wireframe=True,
                          )

r = pydeck.Deck(
    layers=[
        terrain_layer,
        bases_layer, 
        facade_layer, 
        toits_layer
    ],
    initial_view_state=INITIAL_VIEW_STATE,
    map_style='satellite'
)


def filter_by_viewport(widget_instance, payload):
    try:
        west_lng, north_lat = payload['data']['nw']
        east_lng, south_lat = payload['data']['se']
        pad = 0.001
        bbox = [east_lng - pad, south_lat -pad, west_lng+pad, north_lat+pad, ]
        
        f_base, f_face, f_toit = filtered_buildings_geom(bbox)
        txt = f'Buildings in view ({east_lng:.2f} {south_lat:.2f} {west_lng:.2f} {north_lat:.2f}): {f_base.egid.nunique()}'
        if f_base.egid.nunique() > MAX_BUILDINGS:
            # dont try to show too much stuff!
            txt += ' TOO MANY BUILDINGS TO SHOW, TRY ZOOMING IN'
            pass
        else:
        
            bases_layer.data = f_base
            facade_layer.data = f_face
            toits_layer.data = f_toit
        
        text.value = txt
        
        r.update()
    except Exception as e:
        text.value = 'Error: %s' % e

from ipywidgets import HTML

text = HTML(value=f'Buildings in viewport: { f_base.egid.nunique()}')

r.deck_widget.on_view_state_change(filter_by_viewport)
display(text)

r.show()

HTML(value='Buildings in viewport: 36')

DeckGLWidget(carto_key=None, custom_libraries=[], google_maps_key=None, json_input='{"initialViewState": {"lat…